In [25]:
import pandas as pd
import numpy as np
import pymongo
from pymongo import MongoClient
import certifi
from mongo2_config import mongo_uri

In [17]:
Net_hashes_df = pd.read_csv('Mining_NetHashes_cryptocompare_data.csv')
Net_hashes_df

,Unnamed: 0,CoinInfo.Id,Symbol,Name,NetHashesPerSecond,BlockNumber,BlockTime,BlockReward,TotalCoinsMined,AssetLaunchDate,MaxSupply,Price.USD
0,0,1182,BTC,Bitcoin,207280008734314230000,719705,506.000000,6.250000,1.887391e+07,2009-01-03,2.100000e+07,40798.6400
1,1,7605,ETH,Ethereum,977422709818476.4,14045932,13.323235,2.285691,1.183508e+08,2015-07-30,-1.000000e+00,2998.3700
2,9,4432,DOGE,Dogecoin,532399985262592,4070892,63.000000,10000.000000,1.328814e+11,2013-12-06,-1.000000e+00,0.1557
3,8,3808,LTC,Litecoin,364238516146391,2196890,153.000000,12.522045,6.945917e+07,2011-10-13,8.400000e+07,128.9900
4,5,5038,XMR,Monero,3266382349,2541724,116.000000,0.738662,1.806614e+07,2014-04-18,-1.000000e+00,188.8300
5,2,5031,XRP,XRP,0,69150454,0.000000,0.000000,9.998981e+10,2013-01-01,1.000000e+11,0.7256
6,3,171986,USDT,Tether,0,0,0.000000,0.000000,7.827457e+10,2014-10-06,-1.000000e+00,1.0000
7,4,321992,ADA,Cardano,0,6781798,0.000000,0.000000,3.297717e+10,2017-10-05,4.500000e+10,1.2640
8,6,4614,XLM,Stellar,0,39273980,0.000000,0.000000,5.000180e+10,2014-07-31,-1.000000e+00,0.2327
9,7,932135,BUSD,BUSD,0,0,0.000000,0.000000,1.431882e+10,2019-09-10,-1.000000e+00,0.9997


In [18]:
Net_hashes_df = Net_hashes_df.drop(columns=['Unnamed: 0', 'CoinInfo.Id'])
Net_hashes_df.head()

,Symbol,Name,NetHashesPerSecond,BlockNumber,BlockTime,BlockReward,TotalCoinsMined,AssetLaunchDate,MaxSupply,Price.USD
0,BTC,Bitcoin,207280008734314230000,719705,506.000000,6.250000,1.887391e+07,2009-01-03,2.100000e+07,40798.6400
1,ETH,Ethereum,977422709818476.4,14045932,13.323235,2.285691,1.183508e+08,2015-07-30,-1.000000e+00,2998.3700
2,DOGE,Dogecoin,532399985262592,4070892,63.000000,10000.000000,1.328814e+11,2013-12-06,-1.000000e+00,0.1557
3,LTC,Litecoin,364238516146391,2196890,153.000000,12.522045,6.945917e+07,2011-10-13,8.400000e+07,128.9900
4,XMR,Monero,3266382349,2541724,116.000000,0.738662,1.806614e+07,2014-04-18,-1.000000e+00,188.8300


In [20]:
# Rename the column name for the join
Net_hashes_df.rename(columns={'Name': 'CoinName'}, inplace=True)
Net_hashes_df

,Symbol,CoinName,NetHashesPerSecond,BlockNumber,BlockTime,BlockReward,TotalCoinsMined,AssetLaunchDate,MaxSupply,Price.USD
0,BTC,Bitcoin,207280008734314230000,719705,506.000000,6.250000,1.887391e+07,2009-01-03,2.100000e+07,40798.6400
1,ETH,Ethereum,977422709818476.4,14045932,13.323235,2.285691,1.183508e+08,2015-07-30,-1.000000e+00,2998.3700
2,DOGE,Dogecoin,532399985262592,4070892,63.000000,10000.000000,1.328814e+11,2013-12-06,-1.000000e+00,0.1557
3,LTC,Litecoin,364238516146391,2196890,153.000000,12.522045,6.945917e+07,2011-10-13,8.400000e+07,128.9900
4,XMR,Monero,3266382349,2541724,116.000000,0.738662,1.806614e+07,2014-04-18,-1.000000e+00,188.8300
5,XRP,XRP,0,69150454,0.000000,0.000000,9.998981e+10,2013-01-01,1.000000e+11,0.7256
6,USDT,Tether,0,0,0.000000,0.000000,7.827457e+10,2014-10-06,-1.000000e+00,1.0000
7,ADA,Cardano,0,6781798,0.000000,0.000000,3.297717e+10,2017-10-05,4.500000e+10,1.2640
8,XLM,Stellar,0,39273980,0.000000,0.000000,5.000180e+10,2014-07-31,-1.000000e+00,0.2327
9,BUSD,BUSD,0,0,0.000000,0.000000,1.431882e+10,2019-09-10,-1.000000e+00,0.9997


In [11]:
Diff_Profit_mining_df = pd.read_csv('Mining_profit_difficulty_coinwarz_data.csv')
Diff_Profit_mining_df

,Unnamed: 0,index,CoinName,CoinTag,Algorithm,Difficulty,BlockReward,BlockCount,ProfitRatio,AvgProfitRatio,Exchange,ExchangeRate,ExchangeVolume,IsBlockExplorerOnline,IsExchangeOnline,BlockTimeInSeconds,HealthStatus
0,0,0,Peercoin,PPC,SHA-256,2.785680e+09,55.172653,603094,124.616247,127.549330,Bittrex,1.511000e-05,4.078353e+03,True,True,600,Healthy
1,1,1,Ethereum,ETH,EtHash,1.254480e+16,2.000000,14044768,122.951454,129.628541,hitbtc,7.483538e-02,1.454884e+04,True,True,15,Healthy
2,2,2,BitcoinCash,BCH,SHA-256,2.103348e+11,6.250000,723720,103.079621,104.184347,Bittrex,8.810520e-03,1.462686e+02,True,True,600,Healthy
3,3,3,Bitcoin,BTC,SHA-256,2.437187e+13,6.250000,719678,100.000000,100.000000,CoinWarz,4.273544e+04,2.139070e+04,True,True,600,Healthy
4,4,4,Dogecoin,DOGE,Scrypt,4.466169e+06,10000.000000,4070650,67.955174,50.451883,Bittrex,3.830000e-06,5.136060e+06,True,True,60,Healthy
5,5,5,Ethereum-Classic,ETC,ETCHASH,3.110540e+14,3.200000,14373589,57.044417,59.543603,Bittrex,7.360900e-04,6.441766e+02,True,True,15,Healthy
6,6,6,Vertcoin,VTC,Verthash,8.004084e+01,25.000000,1704733,28.324074,30.378896,Bittrex,8.550000e-06,2.234552e+04,True,True,150,Healthy
7,7,7,Ravencoin,RVN,KawPow,7.102181e+04,2500.000000,2113233,17.291629,13.805413,Bittrex,2.000000e-06,4.858957e+06,True,True,60,Healthy
8,8,8,Horizen,ZEN,Equihash,4.438562e+07,3.750000,1076353,16.454699,20.828429,Bittrex,1.205520e-03,3.187807e+02,True,True,150,Healthy
9,9,9,Litecoin,LTC,Scrypt,1.289050e+07,12.500000,2196786,5.922614,6.962510,Bittrex,3.279300e-03,5.244375e+03,True,True,150,Healthy


In [13]:
Diff_Profit_mining_df = Diff_Profit_mining_df.drop(columns=['Unnamed: 0','index'])
Diff_Profit_mining_df.head()

,CoinName,CoinTag,Algorithm,Difficulty,BlockReward,BlockCount,ProfitRatio,AvgProfitRatio,Exchange,ExchangeRate,ExchangeVolume,IsBlockExplorerOnline,IsExchangeOnline,BlockTimeInSeconds,HealthStatus
0,Peercoin,PPC,SHA-256,2.785680e+09,55.172653,603094,124.616247,127.549330,Bittrex,0.000015,4.078353e+03,True,True,600,Healthy
1,Ethereum,ETH,EtHash,1.254480e+16,2.000000,14044768,122.951454,129.628541,hitbtc,0.074835,1.454884e+04,True,True,15,Healthy
2,BitcoinCash,BCH,SHA-256,2.103348e+11,6.250000,723720,103.079621,104.184347,Bittrex,0.008811,1.462686e+02,True,True,600,Healthy
3,Bitcoin,BTC,SHA-256,2.437187e+13,6.250000,719678,100.000000,100.000000,CoinWarz,42735.441600,2.139070e+04,True,True,600,Healthy
4,Dogecoin,DOGE,Scrypt,4.466169e+06,10000.000000,4070650,67.955174,50.451883,Bittrex,0.000004,5.136060e+06,True,True,60,Healthy


In [21]:
Diff_Profit_NetHashes_combine_df = pd.merge(Net_hashes_df, Diff_Profit_mining_df, how="left", on="CoinName")
Diff_Profit_NetHashes_combine_df

,Symbol,CoinName,NetHashesPerSecond,BlockNumber,BlockTime,BlockReward_x,TotalCoinsMined,AssetLaunchDate,MaxSupply,Price.USD,...,BlockCount,ProfitRatio,AvgProfitRatio,Exchange,ExchangeRate,ExchangeVolume,IsBlockExplorerOnline,IsExchangeOnline,BlockTimeInSeconds,HealthStatus
0,BTC,Bitcoin,207280008734314230000,719705,506.000000,6.250000,1.887391e+07,2009-01-03,2.100000e+07,40798.6400,...,719678.0,100.000000,100.000000,CoinWarz,42735.441600,2.139070e+04,True,True,600.0,Healthy
1,ETH,Ethereum,977422709818476.4,14045932,13.323235,2.285691,1.183508e+08,2015-07-30,-1.000000e+00,2998.3700,...,14044768.0,122.951454,129.628541,hitbtc,0.074835,1.454884e+04,True,True,15.0,Healthy
2,DOGE,Dogecoin,532399985262592,4070892,63.000000,10000.000000,1.328814e+11,2013-12-06,-1.000000e+00,0.1557,...,4070650.0,67.955174,50.451883,Bittrex,0.000004,5.136060e+06,True,True,60.0,Healthy
3,LTC,Litecoin,364238516146391,2196890,153.000000,12.522045,6.945917e+07,2011-10-13,8.400000e+07,128.9900,...,2196786.0,5.922614,6.962510,Bittrex,0.003279,5.244375e+03,True,True,150.0,Healthy
4,XMR,Monero,3266382349,2541724,116.000000,0.738662,1.806614e+07,2014-04-18,-1.000000e+00,188.8300,...,2541599.0,-5.327331,-5.374552,hitbtc,0.004756,2.612986e+04,True,True,120.0,Healthy
5,XRP,XRP,0,69150454,0.000000,0.000000,9.998981e+10,2013-01-01,1.000000e+11,0.7256,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,USDT,Tether,0,0,0.000000,0.000000,7.827457e+10,2014-10-06,-1.000000e+00,1.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ADA,Cardano,0,6781798,0.000000,0.000000,3.297717e+10,2017-10-05,4.500000e+10,1.2640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,XLM,Stellar,0,39273980,0.000000,0.000000,5.000180e+10,2014-07-31,-1.000000e+00,0.2327,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BUSD,BUSD,0,0,0.000000,0.000000,1.431882e+10,2019-09-10,-1.000000e+00,0.9997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Exporting data to csv and Uploading on MongoDb

In [24]:
# export the merged df into a csv
Diff_Profit_NetHashes_combine_df.to_csv('Diff_Profit_NetHashes_combined.csv')

In [26]:
# Read in mongodb server location as client
client = MongoClient(mongo_uri,tlsCAFile=certifi.where())

In [27]:
# Find Databases
client.list_database_names()

['Kaggle_data', 'Latest_Cryptodata', 'Latest_StockMarket', 'admin', 'local']

In [28]:
# connecting to the db on MongoDb and creating collection in the db
db = client['Latest_Cryptodata']
collection = db['Diff_Profit_NetHashes_combined_data']

In [29]:
# adding dataframe to mongoDb
Diff_Profit_NetHashes_combine_df.reset_index(inplace=True)
Diff_Profit_NetHashes_combine_df_dict = Diff_Profit_NetHashes_combine_df.to_dict("records")

In [31]:
# Insert collection
collection.insert_many(Diff_Profit_NetHashes_combine_df_dict)